In [1]:
# Import dependencies
import pandas as pd

In [2]:
# Read in .csv data
file = "../Resources/purchase_data.csv"
purchases = pd.read_csv(file)

# Preview dataset
purchases.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count
* Display the total number of players

In [3]:
# Store data frame of unique usernames
uniquePlayers = pd.DataFrame({"Username": purchases["SN"].unique()})

# Calculate & display the total number of players
nPlayers = len(uniquePlayers)
totalPlayers = pd.DataFrame({"Total Players": [nPlayers]})

# Display summary data
totalPlayers

,Total Players
0,576


## Purchasing Analysis (Total)
* Run basic calculations to obtain number of unique items, average price, etc.

* Create a summary data frame to hold the results

* Optional: give the displayed data cleaner formatting

* Display the summary data frame

In [4]:
# Calculate summary data
nItems = len(purchases["Item ID"].unique())

avgPrice = purchases["Price"].mean()

nPurchases = purchases["Purchase ID"].count()

totalRevenue = purchases["Price"].sum()

# Construct the purchasing analysis data frame
purchasingAnalysis = pd.DataFrame({"Number of Unique Items": [nItems],
                                   "Average Price": [avgPrice],
                                   "Number of Purchases": [nPurchases],
                                   "Total Revenue": [totalRevenue]})

In [5]:
# Format avg price and total revenue columns 
purchasingAnalysis["Average Price"] = \
    purchasingAnalysis["Average Price"].astype(float).map("${:,.2f}".format)

purchasingAnalysis["Total Revenue"] = \
    purchasingAnalysis["Total Revenue"].astype(float).map("${:,.2f}".format)

# Display summary data
purchasingAnalysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics
* Percentage and Count of Male Players

* Percentage and Count of Female Players

* Percentage and Count of Other / Non-Disclosed

In [6]:
# Get data frame of number of unique users within each gender
groupedGender = purchases.groupby("Gender")

genderDemographics = pd.DataFrame(groupedGender["SN"].nunique())

# Add column calculating the % of all players that a given gender comprises
genderDemographics["Percentage of Players"] = genderDemographics["SN"]/nPlayers

In [7]:
# Rename count of gender
genderDemographics = genderDemographics.rename(columns={"SN": "Total Players"})

# Format percentage of players column
genderDemographics["Percentage of Players"] = \
    genderDemographics["Percentage of Players"].astype(float).map("{:2.2%}".format)

# Remove index name
# genderDemographics.index.name = None

# Display summary table, sorted by descending % of players
genderDemographics.sort_values(by = ["Percentage of Players"], ascending = False)

,Total Players,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender

* Create a summary data frame to hold the results

* Optional: give the displayed data cleaner formatting

* Display the summary data frame

In [8]:
# Create dataframe with total purchases by gender
genderPurchases = pd.DataFrame(groupedGender["Price"].count())

# Create dataframe with average purchase price by gender
genderRevenue = pd.DataFrame(groupedGender["Price"].mean())

# Join the previous data frames on gender
genderAnalysis = pd.merge(genderPurchases, genderRevenue, on = "Gender", how = "inner")

# Calculate total purchase value
genderAnalysis["Total Purchase Value"] = genderAnalysis["Price_x"] * genderAnalysis["Price_y"]

# Calculate average purchase per person
genderAnalysis["Average Total Purchase per Person"] = \
    [genderAnalysis["Total Purchase Value"][n] / \
        (groupedGender["SN"].nunique()[n]) for n in range(len(genderAnalysis))]

In [9]:
# Rename columns
genderAnalysis = genderAnalysis.rename(columns={"Price_x": "Purchase Count",
                                               "Price_y": "Average Purchase Price"})

# Remove index name
# genderAnalysis.index.name = None

# Format avg price, total value, and avg per person columns
genderAnalysis["Average Purchase Price"] = \
    genderAnalysis["Average Purchase Price"].astype(float).map("${:,.2f}".format)

genderAnalysis["Total Purchase Value"] = \
    genderAnalysis["Total Purchase Value"].astype(float).map("${:,.2f}".format)

genderAnalysis["Average Total Purchase per Person"] = \
    genderAnalysis["Average Total Purchase per Person"].astype(float).map("${:,.2f}".format)

# Display table sorted by descending Purchase Count
genderAnalysis.sort_values(by = "Purchase Count", ascending = False)

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics
* Establish bins for ages

* Categorize the existing players using the age bins. Hint: use pd.cut()

* Calculate the numbers and percentages by age group

* Create a summary data frame to hold the results

* Optional: round the percentage column to two decimal points

* Display Age Demographics Table

In [10]:
# Create bins for age groupings
bins = [0, 9, 14, 19, 24, 29, 34, 39, purchases["Age"].max()]

# Create group names for the bins
groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Create new column with binned age group 
purchases["Age Group"] = pd.cut(purchases["Age"], bins, labels = groups)

In [11]:
# Create object grouped by Age Group
ageGrouped = purchases.groupby("Age Group")

# Create DF with count of unique users within each group
ageDemographics = pd.DataFrame(ageGrouped["SN"].nunique())

# Create column displaying count of users as percentage of all users
ageDemographics["Percentage of Players"] = ageDemographics["SN"]/nPlayers

In [12]:
# Rename columns
ageDemographics = ageDemographics.rename(columns = {"SN": "Total Count"})

# Remove index name
# ageDemographics.index.name = None

# Format percentage of players column
ageDemographics["Percentage of Players"] = \
    ageDemographics["Percentage of Players"].astype(float).map("{:2.2%}".format)

# Display summary data
ageDemographics

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)
* Bin the purchase_data data frame by age

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below

* Create a summary data frame to hold the results

* Optional: give the displayed data cleaner formatting

* Display the summary data frame

In [13]:
# Create dataframe with total purchases by age group
agePurchases = pd.DataFrame(ageGrouped["Price"].count())

# Create dataframe with average purchase price by age group
ageRevenue = pd.DataFrame(ageGrouped["Price"].mean())

# Join the previous dataframes on age group
ageAnalysis = pd.merge(agePurchases, ageRevenue, on = "Age Group", how = "inner")

# Calculate average purchase value 
ageAnalysis["Total Purchase Value"] = ageAnalysis["Price_x"] * ageAnalysis["Price_y"]

# Calculate total purchase per person
ageAnalysis["Average Total Purchase per Person"] = \
    [ageAnalysis["Total Purchase Value"][n] / \
        (ageGrouped["SN"].nunique()[n]) for n in range(len(ageAnalysis))]

In [14]:
# Rename columns
ageAnalysis = ageAnalysis.rename(columns={"Price_x": "Purchase Count",
                                          "Price_y": "Average Purchase Price"})

# Remove index name
# ageAnalysis.index.name = None

# Format avg price, total value, and avg per person columns
ageAnalysis["Average Purchase Price"] = \
    ageAnalysis["Average Purchase Price"].astype(float).map("${:,.2f}".format)

ageAnalysis["Total Purchase Value"] = \
    ageAnalysis["Total Purchase Value"].astype(float).map("${:,.2f}".format)

ageAnalysis["Average Total Purchase per Person"] = \
    ageAnalysis["Average Total Purchase per Person"].astype(float).map("${:,.2f}".format)

# Display summary data
ageAnalysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders
* Run basic calculations to obtain the results in the table below

* Create a summary data frame to hold the results

* Sort the total purchase value column in descending order

* Optional: give the displayed data cleaner formatting

* Display a preview of the summary data frame

In [15]:
# Create object grouped by SN
snGrouped = purchases.groupby("SN")

# Create dataframe with total purchases by SN
snPurchases = snGrouped["Purchase ID"].count()

# Create dataframe with average purchase price by SN
snAvgPurchase = snGrouped["Price"].mean()

# Join the previous dataframes on SN
topSpenders = pd.merge(snPurchases, snAvgPurchase, on = "SN", how = "inner")

# Calculate total purchase value
topSpenders["Total Purchase Value"] = topSpenders["Purchase ID"] * topSpenders["Price"]

In [16]:
# Rename columns
topSpenders = topSpenders.rename(columns={"Purchase ID": "Purchase Count",
                                          "Price": "Average Purchase Price"})

# Sort data by descending purchase value
topSpenders = topSpenders.sort_values(by = "Total Purchase Value", ascending = False)

# Format avg price and total value columns
topSpenders["Average Purchase Price"] = \
    topSpenders["Average Purchase Price"].astype(float).map("${:,.2f}".format)

topSpenders["Total Purchase Value"] = \
    topSpenders["Total Purchase Value"].astype(float).map("${:,.2f}".format)

# Display top five spenders by total purchase value
topSpenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items
* Retrieve the Item ID, Item Name, and Item Price columns

* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value

* Create a summary data frame to hold the results

* Sort the purchase count column in descending order

* Optional: give the displayed data cleaner formatting

* Display a preview of the summary data frame

In [17]:
# Create object grouped by Item ID and Item Name
itemGrouped = purchases.groupby(["Item ID", "Item Name"])

# Create dataframe with total purchases by item
itemPurchases = itemGrouped["Purchase ID"].count()

# Create dataframe with average purchase price by SN
itemAverage = itemGrouped["Price"].mean()

# Join the previous dataframes on item id
topItems = pd.merge(itemPurchases, itemAverage, on = ["Item ID", "Item Name"], how = "inner")

# Calculate total purchase value
topItems["Total Purchase Value"] = topItems["Purchase ID"] * topItems["Price"]

In [18]:
# Rename columns
topItems = topItems.rename(columns={"Purchase ID": "Purchase Count",
                                    "Price": "Item Price"})

# Sort data by descending purchase count
topItems_count = topItems.sort_values(by = "Purchase Count", ascending = False)

# Sort separate data by descending purchase value
topItems_value = topItems.sort_values(by = "Purchase Count", ascending = False)

# Format avg price and total value columns
topItems_count["Item Price"] = \
    topItems_count["Item Price"].astype(float).map("${:,.2f}".format)

topItems_count["Total Purchase Value"] = \
    topItems_count["Total Purchase Value"].astype(float).map("${:,.2f}".format)

# Display top five items by total purchase count
topItems_count.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items
* Sort the above table by total purchase value in descending order

* Optional: give the displayed data cleaner formatting

* Display a preview of the data frame

In [19]:
# Sort data by descending purchase value
topItems_value = topItems_value.sort_values(by = "Total Purchase Value", ascending = False)

# Format avg price and total value columns
topItems_value["Item Price"] = \
    topItems_value["Item Price"].astype(float).map("${:,.2f}".format)

topItems_value["Total Purchase Value"] = \
    topItems_value["Total Purchase Value"].astype(float).map("${:,.2f}".format)

# Display top five items by total purchase value
topItems_value.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
